度学习的话尤其是在CV上都需要对数据做归一化，因为深度神经网络主要就是为了学习训练数据的分布，并在测试集上达到很好的泛化效果，但是，如果我们每一个batch输入的数据都具有不同的分布，显然会给网络的训练带来困难。另一方面，数据经过一层层网络计算后，其数据分布也在发生着变化，此现象称为Internal Covariate Shift  

Internal Covariate Shift ：此术语是google小组在论文Batch Normalizatoin 中提出来的，其主要描述的是：训练深度网络的时候经常发生训练困难的问题，因为，每一次参数迭代更新后，上一层网络的输出数据经过这一层网络计算后，数据的分布会发生变化，为下一层网络的学习带来困难（神经网络本来就是要学习数据的分布，要是分布一直在变，学习就很难了），此现象称之为Internal Covariate Shift  

Covariate Shift 主要描述的是由于训练数据和测试数据存在分布的差异性，给网络的泛化性和训练速度带来了影响，我们经常使用的方法是做归一化或者白化  

##  1. Batch Normalization (BN) 


为了减小Internal Covariate Shift  ，对神经网络的每一层做归一化不就可以了，假设将每一层输出后的数据都归一化到0均值，1方差，满足正太分布，但是，此时有一个问题，每一层的数据分布都是标准正太分布，导致其完全学习不到输入数据的特征，因为，费劲心思学习到的特征分布被归一化了，因此，直接对每一层做归一化显然是不合理的。 
但是如果稍作修改，加入可训练的参数做归一化，那就是实现的了，接下来结合下图的伪代码做详细的分析： 
之所以称之为batchnorm是因为所norm的数据是一个batch的

* 不使用BN，需要小心的调整学习率和权重初始化，用了BN，可以用使用大学习率，从而极大的提高了学习速度  
* Batchnorm本身上也是一种正则的方式，可以代替其他正则方式如dropout等  
* batchnorm降低了数据之间的绝对差异，有一个去相关的性质，更多的考虑相对差异性，因此在分类任务上具有更好的效果。

注：或许大家都知道了，韩国团队在2017NTIRE图像超分辨率中取得了top1的成绩，主要原因竟是去掉了网络中的batchnorm层，由此可见，BN并不是适用于所有任务的，在image-to-image这样的任务中，尤其是超分辨率上，图像的绝对差异显得尤为重要，所以batchnorm的scale并不适合。

In [15]:
import torch
from torch import nn
import numpy as np

x = np.random.rand(5, 3, 2, 2)

# track_running_stats=False，求当前 batch 真实平均值和标准差，而不是更新全局平均值和标准差
# affine=False, 只做归一化，不乘以 gamma 加 beta（通过训练才能确定）
# num_features 为 feature map 的 channel 数目
# eps 设为 0，让官方代码和我们自己的代码结果尽量接近
# bn = nn.BatchNorm2d(num_features=3, eps=0, affine=False, track_running_stats=False)
ch = x.shape[1]
x_ = torch.from_numpy(x).type(torch.FloatTensor)
normal = nn.BatchNorm2d(ch, affine=False, momentum=1)
torch_normal = normal(x_).detach()

print(normal.running_mean)
print(torch.sqrt(normal.running_var))
print(torch_normal[0])

tensor([0.4357, 0.4851, 0.5427])
tensor([0.2864, 0.3270, 0.3229])
tensor([[[-1.4512, -1.0621],
         [-1.3570, -1.1634]],

        [[ 0.8525, -1.1904],
         [ 0.9400,  1.4704]],

        [[ 0.1691, -0.8543],
         [ 0.7979, -1.7138]]])


In [16]:
ch = x.shape[1]
mean = np.mean(x, axis=(0, 2, 3))  # 结果有channel个值， 第一个值是所有样本的第一个channel之和除以 bch * h * w
std = np.std(x, axis=(0, 2, 3))

# mean = np.einsum('ijkl->j', x) / 20
# std = (np.sum((x - mean.reshape(1, ch, 1, 1)) ** 2, axis=(0, 2, 3)) / 20) ** 0.5

print(mean)
print(std)

my_normal = (x - mean.reshape(1, ch, 1, 1)) / std.reshape(1, ch, 1, 1)
print(my_normal[0])

[0.43573084 0.48512855 0.54265071]
[0.2791385  0.31867958 0.31469809]
[[[-1.45133783 -1.06220997]
  [-1.35710962 -1.16350554]]

 [[ 0.85250594 -1.19046422]
  [ 0.94001615  1.47043306]]

 [[ 0.16914378 -0.85435252]
  [ 0.79796884 -1.7138467 ]]]


## 2. Layer Normalization

BN 的一个缺点是需要较大的 batchsize 才能合理估训练数据的均值和方差，这导致内存很可能不够用，同时它也很难应用在训练数据长度不同的 RNN 模型上。Layer Normalization (LN) 的一个优势是不需要批训练，在单条数据内部就能归一化。
batch normalization的缺点：因为统计意义，在batch_size较大时才表现较好；不易用于RNN；训练和预测时用的统计量不同等。  
layer normalization就比较适合用于RNN和单条样本的训练和预测。但是在batch_size较大时性能时比不过batch normalization的。 

LN 针对单个训练样本进行，不依赖于其他数据，因此可以避免 BN 中受 mini-batch 数据分布影响的问题，可以用于 小 mini-batch 场景、动态网络场景和 RNN，特别是自然语言处理领域。此外，LN 不需要保存 mini-batch 的均值和方差，节省了额外的存储空间。 

In [28]:
import torch
from torch import nn
import numpy as np

x = np.random.rand(5, 3, 2, 2)
x_ = torch.from_numpy(x).type(torch.FloatTensor)
bch, inch, h, w = x.shape

normal = nn.LayerNorm(normalized_shape=[inch, h, w])
torch_normal = normal(x_).detach()

print(torch_normal[0])

tensor([[[-1.5256,  0.1740],
         [ 0.1930,  1.1073]],

        [[-1.5460,  0.7371],
         [ 0.7197,  0.3838]],

        [[ 1.1449, -0.8539],
         [-1.4065,  0.8722]]])


In [29]:
mean = np.mean(x, axis=(1, 2, 3))  # 结果有bch个值， 第一个值是第一样本的所有channel之和除以 inch * h * w
std = np.std(x, axis=(1, 2, 3))

# mean = np.sum(x, axis=(1,2,3))/(inch*h*w)
# mean = np.einsum('ijkl->j', x) / 20
# std = (np.sum((x - mean.reshape(1, ch, 1, 1)) ** 2, axis=(0, 2, 3)) / 20) ** 0.5

print(mean)
print(std)

my_normal = (x - mean.reshape(bch, 1, 1, 1)) / std.reshape(bch, 1, 1, 1)
print(my_normal[0])

[0.5454441  0.48788406 0.53163107 0.46335947 0.45032645]
[0.28697367 0.31416167 0.26791705 0.23707632 0.23567182]
[[[-1.52566317  0.17397933]
  [ 0.1930039   1.10739337]]

 [[-1.54611931  0.73711969]
  [ 0.71973291  0.38377823]]

 [[ 1.14501313 -0.85391077]
  [-1.40655153  0.87222422]]]


## 3. Instance Normalization

在图像像素上，对HW做归一化，用在风格化迁移；

In [31]:
import torch
from torch import nn
import numpy as np

x = np.random.rand(5, 3, 2, 2)
x_ = torch.from_numpy(x).type(torch.FloatTensor)
bch, inch, h, w = x.shape

normal = nn.InstanceNorm2d(ch, affine=False, momentum=1)
torch_normal = normal(x_).detach()

print(torch_normal[0])

tensor([[[-1.2670, -0.6650],
         [ 1.1727,  0.7594]],

        [[ 0.6119, -0.0953],
         [ 1.0612, -1.5778]],

        [[ 1.6694, -0.8978],
         [-0.6188, -0.1529]]])


In [33]:

mean = np.mean(x, axis=(2, 3))  
std = np.std(x, axis=(2, 3))

# mean = np.sum(x, axis=(1,2,3))/(inch*h*w)
# mean = np.einsum('ijkl->j', x) / 20
# std = (np.sum((x - mean.reshape(1, ch, 1, 1)) ** 2, axis=(0, 2, 3)) / 20) ** 0.5

# print(mean)
# print(std)

my_normal = (x - mean.reshape(bch, inch, 1, 1)) / std.reshape(bch, inch, 1, 1)
print(my_normal[0])

[[[-1.26711618 -0.66506954]
  [ 1.17276684  0.75941887]]

 [[ 0.61192993 -0.095299  ]
  [ 1.06134474 -1.57797567]]

 [[ 1.66959055 -0.8978543 ]
  [-0.61882372 -0.15291253]]]


当数据的数据in_channel=1时， layer normalization 和instance normalization是一样的

In [40]:
x = np.random.rand(3, 1, 2, 2)
x_ = torch.from_numpy(x).type(torch.FloatTensor)

lnorm = nn.LayerNorm(normalized_shape=[1, 2, 2])
inorm = nn.InstanceNorm2d(ch, affine=False, momentum=1)
print(lnorm(x_)[0])
print(inorm(x_)[0])

tensor([[[-1.6617,  0.9334],
         [ 0.1410,  0.5874]]], grad_fn=<SelectBackward>)
tensor([[[-1.6617,  0.9333],
         [ 0.1410,  0.5874]]])


## 4. Group Normalization

GN介于LN和IN之间，其首先将channel分为许多组（group），对每一组做归一化，及先将feature的维度由[N, C, H, W]reshape为[N, G，C//G , H, W]，归一化的维度为[C//G , H, W]  

## Reference  
https://zhuanlan.zhihu.com/p/69659844  
https://blog.csdn.net/u011511601/article/details/79784960  
https://blog.csdn.net/lx10271129/article/details/78984623  
https://blog.csdn.net/wzy_zju/article/details/81262453 